In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 76kB/s 
     |████████████████████████████████| 204kB 21.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=9c049c31caea51d9ce253c0455f12f73e03b7b6f130d57bf2cd6125eb2dfab7c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
import os
os.chdir('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class')

In [ ]:
from pyspark.sql import SparkSession as ss
spark = ss.builder.getOrCreate()

In [ ]:
files = [file for file in os.listdir()]

In [ ]:
files.remove('Combined Annotated')
files.remove('total_combined_text_and_ids_preprocessed_upd_res')

In [ ]:
files.remove('all_datasets_combined')

In [ ]:
files

['EmoBank_cleaned.csv_preprocessed',
 'SMILE_Twitter_Emotion_dataset_cleaned_preprocessed',
 'Brands_and_Product_Emotions_cleaned.csv_preprocessed',
 'The_Valence_and_Arousal_Facebook_Posts_cleaned_preprocessed',
 'Affect_data_cleaned.csv_preprocessed',
 'Sentiment_Labelled_Sentences_Data_Set_cleaned_preprocessed',
 'Sentiment_Analysis_in_Text_cleaned.csv_preprocessed',
 'Sentiment_Emotion_Mining_Toolkit_(EMTk)_cleaned_preprocessed',
 'Primary_Emotions_of_Statements_cleaned_preprocessed',
 'WASSA_2017_Shared_Task_on_Emotion_Intensity_cleaned.csv_preprocessed',
 'sentiment140-combined']

In [ ]:
dfs = []
for f in files:
  dfs.append(spark.read.csv("/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/"+f,header = True,multiLine=True,inferSchema=True))

In [ ]:
df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10 = dfs

In [ ]:
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

comb_df = unionAll(df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10)

In [ ]:
# comb_df.count()
# comb_df = comb_df.distinct()
# comb_df.count()
from pyspark.sql.functions import sha2, concat_ws
comb_df = comb_df.withColumn("u_id", sha2(concat_ws("||", *comb_df.columns), 256))


# comb_df.write.option("maxRecordsPerFile",10000).option("header","true").csv('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/all_datasets_combined')


In [ ]:
comb_df.show()

+--------------------+--------------------+--------------------+-----------+-----------+-----------+-----------+-----------+----------+--------------------+----------------+--------------------+
|                text|      pre_text_vader|    pre_text_all_upd|vader_score| blob_score|afinn_score|vader_class|afinn_class|blob_class|     mean_sent_score|final_sent_class|                u_id|
+--------------------+--------------------+--------------------+-----------+-----------+-----------+-----------+-----------+----------+--------------------+----------------+--------------------+
|"Read for yoursel...|"Read for yoursel...|read for yourself...|        0.0|        0.0|-0.13333334|        Neu|        Neg|       Neu|-0.04444444666666...|             Neg|a7c4e1b0deda2868f...|
|Photographer kidn...|Photographer kidn...|photographer kidn...|    -0.4404|        0.0|-0.26666668|        Neg|        Neg|       Neu|-0.23568889333333334|             Neg|63782fd8b13d74c60...|
|In return, Gailha...|In 

In [ ]:
comb_df.select('final_sent_class').distinct().rdd.map(lambda r: r[0]).collect()

['Neg', 'VNeg', 'Pos', 'VPos', 'Neu']

In [ ]:
df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12 = dfs

In [ ]:
comb_df = spark.read.csv("/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/all_datasets_combined",header = True,multiLine=True,inferSchema=True)

In [ ]:
from pyspark.sql.functions import sha2, concat_ws
comb_df = comb_df.withColumn("row_sha2", sha2(concat_ws("||", *comb_df.columns), 256))

In [ ]:
comb_df.drop_duplicates().count()

1650302

In [ ]:
comb_df = comb_df.withColumnRenamed('row_sha2','u_id')

In [ ]:
df_fin = comb_df.select("u_id","text","pre_text_vader","pre_text_all_upd","mean_sent_score","final_sent_class")

In [ ]:
df_fin.count()

1648310

In [ ]:
comb_df.write.option("maxRecordsPerFile",10000).option("header","true").csv('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/Combined Annotated/combined_dataset_with_uid')
df_fin.write.option("maxRecordsPerFile",10000).option("header","true").csv('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/Combined Annotated/dataset_final_with_uid')

In [ ]:
files.remove('IMDB_cleaned.csv_preprocessed')

In [ ]:
files

['EmoBank_cleaned.csv_preprocessed',
 'SMILE_Twitter_Emotion_dataset_cleaned_preprocessed',
 'Brands_and_Product_Emotions_cleaned.csv_preprocessed',
 'The_Valence_and_Arousal_Facebook_Posts_cleaned_preprocessed',
 'Affect_data_cleaned.csv_preprocessed',
 'Sentiment_Labelled_Sentences_Data_Set_cleaned_preprocessed',
 'Sentiment_Analysis_in_Text_cleaned.csv_preprocessed',
 'Sentiment_Emotion_Mining_Toolkit_(EMTk)_cleaned_preprocessed',
 'Primary_Emotions_of_Statements_cleaned_preprocessed',
 'WASSA_2017_Shared_Task_on_Emotion_Intensity_cleaned.csv_preprocessed',
 'sentiment140-combined']

In [ ]:
test_df = spark.read.csv("/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/total_combined_text_and_ids_preprocessed_upd_res",header = True,multiLine=True,inferSchema=True)
comb_df = spark.read.csv('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/Combined Annotated/dataset_final_with_uid',header = True,multiLine=True,inferSchema=True)

In [ ]:
test_df.select("text",)

1702455

In [ ]:
comb_df.select('*').subtract(test_df.select('text')).count()

AnalysisException: ignored